In [1]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import scipy.sparse as sp
from collections import defaultdict

mesh = o3d.io.read_triangle_mesh("sound_board.stl")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
vertices = np.asanyarray(mesh.vertices)
triangles = np.asanyarray(mesh.triangles)
print(vertices.shape)
print(triangles.shape)
print(np.min(vertices[:, 0]), np.max(vertices[:, 0]))
print(np.min(vertices[:, 1]), np.max(vertices[:, 1]))
print(np.min(vertices[:, 2]), np.max(vertices[:, 2]))

(235777, 3)
(88576, 3)
-0.1859566569328308 0.18651306629180908
-0.04650379344820976 0.05432426929473877
0.0014315396547317505 0.48692846298217773


In [3]:
def remove_repeat()->None:
    global vertices
    global triangles

    point_to_idx:dict[tuple[float, float, float], int] = dict()

    for (x, y, z) in vertices:
        if (x, y, z) not in point_to_idx:
            point_to_idx[(x, y, z)] = len(point_to_idx)
    
    new_vertices = np.array([list(key) for key in point_to_idx.keys()])
    new_triangles = np.array([[point_to_idx[tuple(vertices[i0])], point_to_idx[tuple(vertices[i1])], point_to_idx[tuple(vertices[i2])]] for (i0, i1, i2) in triangles])
    vertices = new_vertices
    triangles = new_triangles

remove_repeat()
print(vertices.shape)
print(triangles.shape)


(44761, 3)
(88576, 3)


In [4]:
#utils
def make_edge(i0:int, i1:int)->tuple[int, int]:
    return (i0, i1) if i0 < i1 else (i1, i0)

def screen_tris(
        vertices:np.ndarray[tuple[float, float, float]], 
        tris:np.ndarray[tuple[int, int, int]], 
        dim:int,
        thd:float
    )->np.ndarray[tuple[int, int, int]]:

    assert(0 <= dim <= 2)
    index = np.array(range(len(tris)))
    return tris[(vertices[tris[index, 0], dim] < thd) & (vertices[tris[index, 1], dim] < thd) & (vertices[tris[index, 2], dim] < thd)]

def map_vertices(tris:np.ndarray[tuple[int, int, int]], offset:int = 0)->dict[int, int]:
    ret = dict()

    for i0, i1, i2 in tris:
        if i0 not in ret:
            ret[i0] = offset + len(ret)
        if i1 not in ret:
            ret[i1] = offset + len(ret)
        if i2 not in ret:
            ret[i2] = offset + len(ret)

    return ret

def map_edge(
        tris:np.ndarray[tuple[int, int, int]], 
        offset:int
    )->dict[tuple[int, int], int]:

    ret = dict()

    for i0, i1, i2 in tris:
        e0, e1, e2 = make_edge(i0, i1), make_edge(i1, i2), make_edge(i2, i0)
        if e0 not in ret:
            ret[e0] = offset + len(ret)
        if e1 not in ret:
            ret[e1] = offset + len(ret)
        if e2 not in ret:
            ret[e2] = offset + len(ret)
    
    return ret

def reverse_map(d:dict[int, int])->dict[int, int]:
    ret = dict()
    for i in d:
        ret[d[i]] = i
        
    return ret

def dict_create_or_add(d:dict, key:tuple[int, int], v:float)->None:
    if key not in d:
        d[key] = v
    else:
        d[key] += v

f0 = lambda u, v : (1.0 - (u + v) / 2) * (1. - u - v)
f1 = lambda u, v : (1.0 + u) * u / 2.0
f2 = lambda u, v : (1.0 + v) * v / 2.0
f3 = lambda u, v : 4.0 * u * (1.0 - u - v)
f4 = lambda u, v : 4 * u * v
f5 = lambda u, v : 4 * v * (1.0 - u - v)
f6 = lambda u, v : 27 * u * v * (1.0 - u - v)

f = [f0, f1, f2, f3, f4, f5, f6]

def get_coef(v0, v1, p)->tuple[float, float]:
    u = np.dot(v0, p) / np.linalg.norm(v0)
    v = np.dot(v1, p) / np.linalg.norm(v1)

    if u >= 0 and v >= 0 and (u + v) <= 1:
        return (u, v)
    else:
        return False


def get_boundary_edges(
        vertices:np.ndarray[tuple[float, float, float]], 
        tris:np.ndarray[tuple[float, float, float]], 
        axis0:int, 
        min0:float, 
        max0:float, 
        axis1:int, 
        min1:float, 
        max1:float, 
        is_inner:bool)->list[tuple[int, int, int]]:
    #@return <edge_vertice_idx0, edge_vertice_idx1, tri_idx>

    edge_map:dict[tuple[int, int], list[int]] = defaultdict(lambda : [])
    #edge_map.key <edge_vertice_idx0, edge_vertice_idx1> such that edge_vertice_idx0 < edge_vertice_idx1
    #edge_map.value <list[tri_idx]>
    judge_func = lambda cord3d : (min0 <= cord3d[axis0] <= max0 and min1 <= cord3d[axis1] <= max1) if is_inner else (cord3d[axis0] < min0 or cord3d[axis0] > max0 or cord3d[axis1] < min1 or cord3d[axis1] > max1)

    for tri_idx, (i0, i1, i2) in enumerate(tris):
        edge_map[make_edge(i0, i1)].append(tri_idx)
        edge_map[make_edge(i1, i2)].append(tri_idx)
        edge_map[make_edge(i0, i2)].append(tri_idx)
    
    ret = []
    for idx0, idx1 in edge_map:
        if len(tmp:=edge_map[(idx0, idx1)]) == 1 and judge_func(vertices[idx0]) and judge_func(vertices[idx1]):
            ret.append((idx0, idx1, tmp[0]))
    return ret

def adapt_one_tri(
    tris:np.ndarray[tuple[int, int, int]], 
    idx:int, 
    target_i0:int, 
    target_i1:int
)->None:
    assert target_i0 in tris[idx], f"{target_i0} must in tris[{idx}]"
    assert target_i1 in tris[idx], f"{target_i1} must in tris[{idx}]"
    i2 = None
    for i in tris[idx]:
        if i != target_i0 and i != target_i1:
            i2 = i
            break
    tris[idx][0] = target_i0
    tris[idx][1] = target_i1
    tris[idx][2] = i2

def reorder_tris(        
        tris:np.ndarray[tuple[int, int, int]], 
        boundary:list[tuple[int, int, int]], 
        ref_dir:bool = True)->list[tuple[int, int, int]]:
    #@return list[<edge_start, edge_end, tri_idx>]
    edge_map:dict[int, list[tuple[int, int]]] = defaultdict(lambda : [])
    #edge_map.key <edge_vertice_idx>
    #edge_map.value <list[<edge_vertice_neighbor, tri_idx>]>

    for i0, i1, tri_idx in boundary:
        edge_map[i0].append((i1, tri_idx))
        edge_map[i1].append((i0, tri_idx))

    vis:set = set()
    ret:list[int] = []
    head = None
    next = None
    if ref_dir:
        head = boundary[0][0]
        next = boundary[0][1]
    else:
        head = boundary[0][1]
        next = boundary[0][0]
    vis.add(head)
    ret.append((head, next, boundary[0][2]))

    while next not in vis:
        (i0, tri0), (i1, tri1) = edge_map[next]
        vis.add(next)
        if i0 not in vis:
            ret.append((next, i0, tri0))
            next = i0
        if i1 not in vis:
            ret.append((next, i1, tri1))
            next = i1
        
    (i0, tri0), (i1, tri1) = edge_map[next]
    if i0 == head:
        ret.append((next, head, tri0))
    if i1 == head:
        ret.append((next, head, tri1))

    return ret

def create_sprase_mat(dict:defaultdict[tuple[int, int], float], shape:tuple[int, int])->sp.coo_matrix:
    row_idxs:list[int] = []
    col_idxs:list[int] = []
    vals:list[float] = []

    for r_idx, c_idx in dict:
        row_idxs.append(r_idx)
        col_idxs.append(c_idx)
        vals.append(dict[(r_idx, c_idx)])
    
    return sp.coo_matrix((vals, (row_idxs, col_idxs)), shape=shape)
    

In [5]:
#vis utils
import open3d as o3d
import numpy as np
X_AXIS = o3d.geometry.TriangleMesh.create_arrow(
        cylinder_radius=0.0005, cone_radius=0.001, cylinder_height=0.008, cone_height=0.002
    )
X_AXIS.compute_vertex_normals()
X_AXIS.paint_uniform_color([1.0, 0.0, 0.0])
X_AXIS.rotate(o3d.geometry.get_rotation_matrix_from_axis_angle(np.array([0.0, np.pi / 2, 0.0]).reshape(3, 1)), center = (0,0,0))
Y_AXIS = o3d.geometry.TriangleMesh.create_arrow(
        cylinder_radius=0.0005, cone_radius=0.001, cylinder_height=0.008, cone_height=0.002
    )
Y_AXIS.compute_vertex_normals()
Y_AXIS.paint_uniform_color([0.0, 1.0, 0.0])
Y_AXIS.rotate(o3d.geometry.get_rotation_matrix_from_axis_angle(np.array([np.pi * 1.5, 0.0, 0.0]).reshape(3, 1)), center = (0,0,0))
Z_AXIS = o3d.geometry.TriangleMesh.create_arrow(
        cylinder_radius=0.0005, cone_radius=0.001, cylinder_height=0.008, cone_height=0.002
    )
Z_AXIS.compute_vertex_normals()
Z_AXIS.paint_uniform_color([0.0, 0.0, 1.0])


def show_vector(start:tuple[float, float, float], end:tuple[float, float, float], color:tuple[float, float, float])->o3d.geometry:
    start = np.array(start)
    end = np.array(end)
    dir = end - start
    l_dir = np.linalg.norm(dir)
    e_dir = dir / l_dir

    cylinder_height = l_dir * 0.6
    cone_height = l_dir * 0.4
    cylinder_radius = cylinder_height / 16
    cone_radius = cone_height / 2
    ret = o3d.geometry.TriangleMesh.create_arrow(
        cylinder_radius=cylinder_radius, cone_radius=cone_radius, cylinder_height=cylinder_height, cone_height=cone_height
    )
    ret.compute_vertex_normals()
    ret.paint_uniform_color(color)
    rot_axis = np.cross(np.array([0.0, 0.0, 1.0]), e_dir)
    rot_axis /= np.linalg.norm(rot_axis)
    cos_theta = np.dot(np.array([0.0, 0.0, 1.0]), e_dir) #since unit vector
    theta = np.arccos(cos_theta)
    theta += 2 *np.pi
    ret.rotate(o3d.geometry.get_rotation_matrix_from_axis_angle(rot_axis.reshape(3,1) * theta), center = (0,0,0))
    ret.translate(start)
    return ret

In [6]:
config:dict = {
    "D1":850.0, 
    "D2":50.0, 
    "D3":75.0, 
    "D4":200.0, 
    "plate vertical axis":1, 
    "plate thd":-0.04645, 
    "plate x axis":0, 
    "plate x min":-0.075,
    "plate x max":0.075,
    "plate y axis":2, 
    "plate y min":0.25,
    "plate y max":0.40, 
    "gamma_0_ref_dir":False, 
    "gamma_f_ref_dir":False, 
    "x0":0.0, 
    "y0":0.28
}


class SoundBoard:
    def __init__(
            self, 
            vertices:np.ndarray[tuple[float, float, float]], 
            tris:np.ndarray[tuple[int, int, int]], 
            config:dict
        ) -> None:
        print("SoundBoard Info:")
        self.vertices = vertices
        inner_tris:np.ndarray[tuple[int, int, int]] = screen_tris(self.vertices, tris, config["plate vertical axis"], config["plate thd"])
        self.vertices_map = map_vertices(inner_tris)
        self.num_vertices = len(self.vertices_map)
        self.edge_map = map_edge(inner_tris, self.num_vertices)
        self.num_edges = len(self.edge_map)
        self.num_centers = len(inner_tris)
        self.config = config
        self.data = np.array([(Idx_tri_0, Idx_tri_1, Idx_tri_2, self.vertices_map[Idx_tri_0], self.vertices_map[Idx_tri_1], self.vertices_map[Idx_tri_2], 
                      self.edge_map[make_edge(Idx_tri_0, Idx_tri_1)], self.edge_map[make_edge(Idx_tri_1, Idx_tri_2)], self.edge_map[make_edge(Idx_tri_0, Idx_tri_2)], 
                      self.num_vertices + self.num_edges + i) for i, (Idx_tri_0, Idx_tri_1, Idx_tri_2) in enumerate(inner_tris)])
        self.gamma_f = reorder_tris(inner_tris, get_boundary_edges(vertices, inner_tris, config["plate x axis"], config["plate x min"], config["plate x max"], config["plate y axis"], config["plate y min"], config["plate y max"], True), config["gamma_f_ref_dir"])
        self.gamma_0 = reorder_tris(inner_tris, get_boundary_edges(vertices, inner_tris, config["plate x axis"], config["plate x min"], config["plate x max"], config["plate y axis"], config["plate y min"], config["plate y max"], False), config["gamma_0_ref_dir"] )
        
        print(f"# of vertices: {self.num_vertices}")
        print(f"# of edges: {self.num_edges}")
        print(f"# of centriods: {self.num_centers}")
        print(f"# of total points: {self.total_points()}")
        print(f"# of edges on gamma_f: {len(self.gamma_f)}")
        print(f"# of edges on gamma_0: {len(self.gamma_0)}")
        self.__get_M_Mh()
        self.__get_H_h()

    
    def total_points(self)->int:
        return self.num_vertices + self.num_edges + self.num_centers

    
    def get_gamma_f_edges(self, color:tuple[float, float, float] = (0.2, 0.2, 0.7))->list[o3d.geometry]:
        ret = []
        for idx_start, idx_end, tri_idx in self.gamma_f:
            ret.append(show_vector(self.vertices[idx_start], self.vertices[idx_end], color))
        return ret

    
    def get_gamma_0_edges(self, color:tuple[float, float, float] = (0.2, 0.2, 0.7))->list[o3d.geometry]:
        ret = []
        for idx_start, idx_end, tri_idx in self.gamma_0:
            ret.append(show_vector(self.vertices[idx_start], self.vertices[idx_end], color))
        return ret
    


    def __get_M_ph(self)->None:
        M_int = [
            [13 / 240, 29 / 1440, 29 / 1440, 1 / 20, 1 / 45, 1 / 20, 3 / 56], 
            [29 / 1440, 13 / 240, 29 / 1440, 1 / 20, 1 / 20, 1 / 45, 3 / 56],
            [29 / 1440, 29 / 1440, 13 / 240, 1 / 45, 1 / 20, 1 / 20, 3 / 56],
            [1 / 20, 1 / 20, 1 / 45, 4 / 45, 2 / 45, 2 / 45, 2 / 45, 3 / 35],
            [1 / 45, 1 / 20, 1 / 20, 2 / 45, 4 / 45, 2 / 45, 3 / 35], 
            [1 / 20, 1 / 45, 1 / 20, 2 / 45, 2 / 45, 4 / 45, 3 / 35], 
            [3 / 56, 3 / 56, 3 / 56, 3 / 35, 3 / 35, 3 / 35, 81 / 560]
        ]

        data_dict:defaultdict[tuple[int, int], float] = defaultdict(lambda : 0.0)
        a_p = self.config["a_p"]
        rho_p = self.config["rho_p"]

        for loc_i0, loc_i1, loc_i2, *global_idxs in self.data:
            p0 = self.vertices[loc_i0]
            p1 = self.vertices[loc_i1]
            p2 = self.vertices[loc_i2]

            vec_u = p1 - p0
            vec_v = p2 - p0
            vec_u[1] = 0.0
            vec_v[1] = 0.0
            u_cross_v = np.cross(vec_u, vec_v)
            coef = a_p * rho_p * np.linalg.norm(u_cross_v)

            for i in range(7):
                for j in range(7):
                    data_dict[(global_idxs[j], global_idxs[i])] += coef * M_int[i][j]

        self.M_ph = create_sprase_mat(data_dict, (self.total_points(), self.total_points()))

    def __get_J_h(self)->None:
        x0 = self.config["x0"]
        y0 = self.config["y0"]
        Ns = self.config["Ns"]
        T  = self.config["T"]
        data_dict:defaultdict[tuple[int, int], float] = defaultdict(lambda : 0.0)
        cnt = 0
        for loc_i0, loc_i1, loc_i2, *global_idxs in self.data:
            p0 = self.vertices[loc_i0]
            p1 = self.vertices[loc_i1]
            p2 = self.vertices[loc_i2]

            vec_target = np.array([x0, 0, y0]) - p0
            vec_u = p1 - p0
            vec_v = p2 - p0
            vec_target[1] = 0.0
            vec_u[1] = 0.0
            vec_v[1] = 0.0

            u = np.dot(vec_u, vec_target) / np.dot(vec_u, vec_u)
            v = np.dot(vec_v, vec_target) / np.dot(vec_v, vec_v)

            if u >= 0.0 and v >= 0.0 and u + v <= 1.0:
                cnt += 1
                for i in range(7):
                    data_dict[(global_idxs[i], Ns-1)] += f[i](u, v) * T

        assert cnt == 1, "only one satisfied triangle"
        self.J_h = create_sprase_mat(data_dict, (self.total_points(), Ns))
 
    def __get_M_Mh(self)->None:
        M_int = [
            [ 13 / 240, 29 / 1440, 29 / 1440, 1 / 20, 1 / 45, 1 / 20, 3 / 56], 
            [ 29 / 1440, 13 / 240, 29 / 1440, 1 / 20, 1 / 20, 1 / 45, 3 / 56], 
            [ 29 / 1440, 29 / 1440, 13 / 240, 1 / 45, 1 / 20, 1 / 20, 3 / 56], 
            [ 1 / 20, 1 / 20, 1 / 45, 4 / 45, 2 / 45, 2 / 45, 3 / 35], 
            [ 1 / 45, 1 / 20, 1 / 20, 2 / 45, 4 / 45, 2 / 45, 3 / 35], 
            [ 1 / 20, 1 / 45, 1 / 20, 2 / 45, 2 / 45, 4 / 45, 3 / 35], 
            [3 / 56, 3 / 56, 3 / 56, 3 / 35, 3 / 35, 3 / 35, 81 / 560]
        ]

        D1 = self.config["D1"]
        D2 = self.config["D2"]
        D3 = self.config["D3"]
        D4 = self.config["D4"]

        inv_det = 1 / (D1 * D3 - D2 * D2 / 4)

        inv_C = [
            [D3 * inv_det     ,-D2 * inv_det / 2 ,   0   ], 
            [-D2 * inv_det / 2 ,D1 * inv_det     ,   0   ],
            [               0 ,                0, 4 / D4]
        ]

        data:dict[tuple[int, int], float] = defaultdict(lambda : 0.0)

        for loc_i0, loc_i1, loc_i2, *global_idx in self.data:
            loc_p0 = self.vertices[loc_i0]
            loc_p1 = self.vertices[loc_i1]
            loc_p2 = self.vertices[loc_i2]

            vec_u = loc_p1 - loc_p0
            vec_v = loc_p2 - loc_p0
            vec_u[1] = 0.0
            vec_v[1] = 0.0
            norm_cross_uv = np.linalg.norm(np.cross(vec_u, vec_v))

            for i in range(7):
                for j in range(i, 7):
                    int_v = M_int[i][j] * norm_cross_uv
                    for ii in range(3):
                        for jj in range(3):
                            b_ii_jj = inv_C[ii][jj]
                            data[(3 * global_idx[i] + ii, 3 * global_idx[j] + jj)] += int_v * b_ii_jj

        row_idxs:list[int] = []
        col_idxs:list[int] = []
        vals:list[float]   = []

        for r_idx, c_idx in data:
            row_idxs.append(r_idx)
            col_idxs.append(c_idx)
            vals.append(data[(r_idx, c_idx)])

        self.M_Mh = sp.coo_matrix((vals, (row_idxs, col_idxs)), shape=(self.total_points() * 3, self.total_points() * 3))

    def __get_H_h(self)->None:
        M_int_uu = [
            [3 / 8, -1 / 3, 0, -1 / 6, -1 / 2, 1 / 2, -9 / 40], 
            [-1 / 3 , 3 / 8, 0, -1 / 6, 1 / 2, -1 / 2, -9 / 40], 
            [0, 0, 0, 0, 0, 0, 0], 
            [-1 / 6, -1 / 6, 0, 4 / 3, 0, 0, 9 / 5], 
            [-1 / 2, 1 / 2, 0, 0, 4 / 3, -4 / 3, 0], 
            [1 / 2, -1 / 2, 0, 0, -4 / 3, 4 / 3, 0], 
            [-9 / 40, -9 / 40, 0, 9 / 5, 0, 0, 81 / 20]
        ]

        M_int_uv = [
            [3 / 8, 0, -1 / 3, 1 / 2, -1 / 2, -1 / 6, -9 / 40], 
            [-1 / 3, 0, 1 / 3, -2 / 3, 2 / 3, 0, 0], 
            [0, 0, 0, 0, 0, 0, 0], 
            [-1 / 6, 0, 0, 2 / 3, -2 / 3, 2 / 3, 9 / 10], 
            [-1 / 2, 0, 2 / 3, -2 / 3, 2 / 3, -2 / 3, -9 / 10], 
            [1 / 2, 0, -2 / 3, 2 / 3, -2 / 3, 2 / 3, 9 / 10], 
            [-9 / 40, 0, 0, 9 / 10, -9 / 10, 9 / 10, 81 / 40]
        ]

        M_int_vv = [
            [3 / 8, 0, -1 / 3, 1 / 2, -1 / 2, -1 / 6, -9 / 40], 
            [0, 0, 0, 0, 0, 0, 0], 
            [-1 / 3, 0, 3 / 8, -1 / 2, 1 / 2, -1 / 6, -9 / 40], 
            [1 / 2, 0, -1 / 2, 4 / 3, -4 / 3, 0, 0], 
            [-1 / 2, 0, 1 / 2, -4 / 3, 4 / 3, 0, 0], 
            [-1 / 6, 0, -1 / 6, 0, 0, 4 / 3, 9 / 5], 
            [-9 / 40, 0, -9 / 40, 0, 0, 9 / 5, 81 / 20]
        ]

        base_func_map:dict[tuple[int, int], tuple[tuple[int, int, int], tuple[int, int, int]]] = {
            (0, 1):([0, 1, 2], [0, 1, 2]), 
            (1, 0):([1, 0, 2], [0, 2, 1]), 
            (0, 2):([0, 2, 1], [2, 1, 0]), 
            (2, 0):([2, 0, 1], [2, 0, 1]), 
            (1, 2):([1, 2, 0], [1, 2, 0]), 
            (2, 1):([2, 1, 0], [1, 0, 2])
        }

        M_int_u = [
            [-1 / 2, 1 / 3, 0, 2 / 3, 0, 0, 0], 
            [-1 / 3, 1 / 2, 0, -2 / 3, 0, 0, 0], 
            [0, 0, 0, 0, 0, 0, 0], 
            [-2 / 3, 2 / 3, 0, 0, 0, 0, 0], 
            [0, 0, 0, 0, 0, 0, 0], 
            [0, 0, 0, 0, 0, 0, 0], 
            [0, 0, 0, 0, 0, 0, 0]
        ]

        data:defaultdict[tuple[int, int], float] = defaultdict(lambda : 0.0)

        #plate 

        for loc_i0, loc_i1, loc_i2, *global_idx in self.data:

            loc_p0 = self.vertices[loc_i0]
            loc_p1 = self.vertices[loc_i1]
            loc_p2 = self.vertices[loc_i2]

            vec_u = loc_p1 - loc_p0
            vec_v = loc_p2 - loc_p0
            vec_u[1] = 0.0
            vec_v[1] = 0.0
            norm_corss_uv = np.linalg.norm(np.cross(vec_u, vec_v))
            Q = np.array([
                [vec_u[0], vec_v[0]], 
                [vec_u[2], vec_v[2]]
            ])
            inv_Q = np.linalg.inv(Q)

            for i in range(7):
                for j in range(7):
                    data[(3 * global_idx[j] + 0, global_idx[i])] += norm_corss_uv * (M_int_uu[i][j] * inv_Q[0, 0] * inv_Q[0, 0] + (M_int_uv[i][j] + M_int_uv[j][i]) * inv_Q[0, 0] * inv_Q[1, 0] + M_int_vv[i][j] * inv_Q[1, 0] * inv_Q[1, 0])
                    data[(3 * global_idx[j] + 1, global_idx[i])] += norm_corss_uv * (M_int_uu[i][j] * inv_Q[0, 1] * inv_Q[0, 1] + (M_int_uv[i][j] + M_int_uv[j][i]) * inv_Q[0, 1] * inv_Q[1, 1] + M_int_vv[i][j] * inv_Q[1, 1] * inv_Q[1, 1])
                    data[(3 * global_idx[j] + 2, global_idx[i])] += norm_corss_uv * (2.0 * M_int_uu[i][j] * inv_Q[0, 0] * inv_Q[0, 1] + (M_int_uv[i][j] + M_int_uv[j][i]) * (inv_Q[0, 0] * inv_Q[1, 1] + inv_Q[0, 1] * inv_Q[1, 0]) + 2.0 * M_int_vv[i][j] * inv_Q[1, 0] * inv_Q[1, 1])

        #gamma_f
        for loc_i0, loc_i1 , tri_idx in self.gamma_f:
            tri_data = self.data[tri_idx]
            loc_idxs, global_idx_012, global_idx_345 = list(tri_data[:3]), np.array(tri_data[3:6]), np.array(tri_data[6:9])
            transform_012, transform_345 = base_func_map[(loc_idxs.index(loc_i0), loc_idxs.index(loc_i1))]
            global_idx_012 = global_idx_012[transform_012]
            global_idx_345 = global_idx_345[transform_345]

            vec_tao = self.vertices[loc_i1] - self.vertices[loc_i0]
            vec_tao[1] = 0.0
            inv_len_vec_tao = 1. / np.linalg.norm(vec_tao)

            vec_n = np.cross(vec_tao, [0.0, 1.0, 0.0])
            vec_n /= np.linalg.norm(vec_n)

            global_idx = list(tri_data[3:])
            tri_data[3:6] = list(global_idx_012)
            tri_data[6:9] = list(global_idx_345)

            for j in range(7):
                for i in [0, 1, 3]:
                    data[(3 * global_idx[j] + 0, global_idx[i])] += inv_len_vec_tao * vec_n[0] * vec_tao[0] * M_int_u[i][j]
                    data[(3 * global_idx[j] + 1, global_idx[i])] += inv_len_vec_tao * vec_n[2] * vec_tao[2] * M_int_u[i][j]
                    data[(3 * global_idx[j] + 2, global_idx[i])] += inv_len_vec_tao * (vec_n[0] * vec_tao[2] + vec_n[2] * vec_tao[0]) * M_int_u[i][j]


        row_idxs:list[int] = []
        col_idxs:list[int] = []
        vals:list[float]   = []

        for r_idx, c_idx in data:
            row_idxs.append(r_idx)
            col_idxs.append(c_idx)
            vals.append(data[(r_idx, c_idx)])

        self.M_Mh = sp.coo_matrix((vals, (row_idxs, col_idxs)), shape=(self.total_points() * 3, self.total_points()))



    def __get_B_w_h_transpose(self):
        M_int_gf:list[list[float]] = [
            [1 / 15, 7 / 240, 7 / 240, 1 / 15, 1 / 30, 1 / 15, 3 / 40],
            [23 / 240, 23 / 240, 7 / 120, 2 / 15, 1 / 10, 1 / 10, 3 / 20], 
            [23 / 240, 7 / 120, 23 / 240, 1 / 10, 1 / 10, 2 /15, 3 / 20]
        ]

        data:defaultdict[tuple[int, int], float] = defaultdict(lambda : 0.0)

        for loc_i0, loc_i1, loc_i2, *global_idxs in self.data:
            p0 = self.vertices[loc_i0]
            p1 = self.vertices[loc_i1]
            p2 = self.vertices[loc_i2]

            vec_u = p1 - p0
            vec_v = p2 - p0
            vec_u[1] = 0.0
            vec_v[1] = 0.0

            u_cross_v = np.cross(vec_u, vec_v)
            len_u_cross_v = np.linalg.norm(u_cross_v)

            for i in range(7):
                for j in range(3):
                    data[(global_idxs[i], global_idxs[j])] += len_u_cross_v * M_int_gf[j][i]

        self.B_w_h_transpose = create_sprase_mat(data, (self.total_points(), len(self.vertices)))

In [7]:
#Test
sd = SoundBoard(vertices, triangles, config)

SoundBoard Info:
# of vertices: 3508
# of edges: 10200
# of centriods: 6692
# of total points: 20400
# of edges on gamma_f: 112
# of edges on gamma_0: 212


In [8]:
o3d.visualization.draw_geometries([X_AXIS, Y_AXIS, Z_AXIS] + sd.get_gamma_f_edges())